In [ ]:
# Allow reload of objects
%load_ext autoreload
%autoreload
from pelenet.utils import Utils
from pelenet.experiments.assembly import Experiment

# Official modules
import numpy as np
import nxsdk.api.n2a as nx
import matplotlib
import matplotlib.pyplot as plt
from time import time

In [ ]:
# Initilizes the experiment and utils
utils = Utils.instance()
exp = Experiment()

In [ ]:
# Build network
exp.build()

# Get spectral radius
utils.getSpectralRadius(exp.net.initialWeights)

In [ ]:
exp.run()